<a href="https://colab.research.google.com/github/StevenDreamer1/DATA-MAINING/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
# Load dataset from TensorFlow Datasets (No manual download required)
dataset_name = "cats_vs_dogs"
dataset, info = tfds.load(dataset_name, as_supervised=True, with_info=True)
# Split dataset into training and validation
train_data = dataset['train'].take(5000) # First 20,000 for training
val_data = dataset['train'].skip(5000).take(1000) # Next 5,000 for validation
# Function to preprocess images (resize, normalize)
def preprocess(image, label):
 image = tf.image.resize(image, (224, 224)) # Resize to VGG16 expected size
 image = image / 255.0 # Normalize to [0,1]
 return image, label
# Apply preprocessing and batching
train_data = train_data.map(preprocess).batch(128).shuffle(1000)
val_data = val_data.map(preprocess).batch(128)
# Load Pre-trained VGG16 Model (without top layers)
base_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3),
 include_top=False, weights='imagenet')
# Freeze the base model (so pre-trained weights are not changed)
base_model.trainable = False
# Add custom classifier on top
model = tf.keras.Sequential([
 base_model,
 tf.keras.layers.Flatten(),
 tf.keras.layers.Dense(256, activation='relu'),
 tf.keras.layers.Dropout(0.5),
 tf.keras.layers.Dense(1, activation='sigmoid') # Binary classification
])
# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Train Model
history = model.fit(train_data, validation_data=val_data, epochs=3)
# Evaluate Model
loss, acc = model.evaluate(val_data)
print(f"\nValidation Accuracy: {acc * 100:.2f}%")
# Function to display a predicted image
def show_prediction():
 image, label = next(iter(val_data)) # Get a batch
 img = image[0].numpy() # Convert tensor to numpy array
 true_label = label[0].numpy()

 prediction = model.predict(tf.expand_dims(image[0], axis=0))
 predicted_label = "Dog" if prediction[0][0] > 0.5 else "Cat"
 plt.imshow(img)
 plt.title(f"Predicted: {predicted_label}, Actual: {'Dog' if true_label else 'Cat'}")
 plt.axis("off")
 plt.show()
# Show a random predicted image
show_prediction()

Epoch 1/3
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 66s/step - accuracy: 0.5799 - loss: 2.7659  